# Compare ACE2 binding vs serum escape

In [1]:
import altair as alt

import pandas as pd

## Format summary dataframe

In [2]:
# read in data
XBB15_mACE2_all = (pd.read_csv("results/summaries/summary.csv"))
# make mutation column
XBB15_mACE2_all['mutation'] = XBB15_mACE2_all['wildtype']+XBB15_mACE2_all['site'].astype(str)+XBB15_mACE2_all['mutant']
# get rid of anything that has NAs or functional score with <=-1.5
XBB15_mACE2_all = XBB15_mACE2_all.dropna(
    subset=['human sera escape', 'ACE2 binding']).loc[XBB15_mACE2_all['spike mediated entry'] >= -1.5]

## Calculate R between ACE2 binding and affinity for each site

In [3]:
positions = XBB15_mACE2_all['site'].unique()
correlation_data = []

for position in positions:
    subset = XBB15_mACE2_all[XBB15_mACE2_all['site'] == position]
    correlation = subset[['human sera escape', 'ACE2 binding']].corr().iloc[0, 1]
    correlation_data.append({'site': position, 'correlation_text': f"R = {correlation:.2f}", 'correlation': correlation})

correlation_df = pd.DataFrame(correlation_data) # sites with only WT have NaN
correlation_df = correlation_df.dropna(subset=['correlation'])


In [4]:
# add correlation to binding/escape df
XBB15_df = pd.merge(XBB15_mACE2_all, correlation_df,
                          how='left',
                          on=['site'],
                         ).dropna(subset=['correlation'])

In [11]:
## df for sites with high correlation
site_counts = XBB15_df['site'].value_counts()

# filter sites with >= 9 mutations measured
mask = XBB15_df['site'].isin(site_counts[site_counts >= 7].index)

# Apply the mask to filter the DataFrame
filtered_XBB15_df = XBB15_df[mask]

# get sites with R<=-0.82
high_correlation= filtered_XBB15_df.loc[filtered_XBB15_df['correlation'] <= -0.82]

In [12]:
XBB15_df.loc[XBB15_df['site'] == 570]


,site,wildtype,mutant,human sera escape,spike mediated entry,ACE2 binding,sequential_site,region,mutation,correlation_text,correlation
3729,570,A,D,-0.57490,-1.37500,1.234000,566,other,A570D,R = -0.97,-0.972861
3730,570,A,G,-0.32030,-0.56760,0.578700,566,other,A570G,R = -0.97,-0.972861
3731,570,A,S,-0.03300,0.09871,0.009861,566,other,A570S,R = -0.97,-0.972861
3732,570,A,T,0.45610,0.09598,-0.550600,566,other,A570T,R = -0.97,-0.972861
3733,570,A,V,-0.06467,-0.02958,0.046520,566,other,A570V,R = -0.97,-0.972861
3734,570,A,Y,-0.53410,-1.09400,1.274000,566,other,A570Y,R = -0.97,-0.972861
3735,570,A,A,0.00000,0.00000,0.000000,566,other,A570A,R = -0.97,-0.972861


## Plot sites with high inverse correlation

In [13]:
init_min_func_effect = -1.5

func_effects_slider = alt.param(
    value=init_min_func_effect,
    bind=alt.binding_range(
        name="minimum spike-mediated entry",
        min=high_correlation["spike mediated entry"].min(),
        max=0,
    ),
)

affinity_escape_corr_base = (
    alt.Chart(high_correlation)
    .add_params(func_effects_slider)
    .transform_filter(alt.datum["spike mediated entry"] >= func_effects_slider)
)


affinity_escape_corr_chart = (
    (
        (
            affinity_escape_corr_base
            .encode(
                x=alt.X("ACE2 binding", scale=alt.Scale(nice=False, padding=10)),
                y=alt.Y("human sera escape", scale=alt.Scale(nice=False, padding=10)),
                tooltip=high_correlation.columns.tolist(),
            )           
            .mark_circle(color="black", opacity=0.25, size=120)
        )
        + (
            affinity_escape_corr_base
            .transform_regression("ACE2 binding", "human sera escape", params=True)
            .transform_calculate(
                r=alt.expr.if_(
                    alt.datum["coef"][1] > 0,
                    alt.expr.sqrt(alt.datum["rSquared"]),
                    -alt.expr.sqrt(alt.datum["rSquared"]),
                ),
                r_text="r = " + alt.expr.format(alt.datum["r"], ".2f"),
            )
            .encode(
                text="r_text:N",
                x=alt.value(5),
                y=alt.value(170),
            )
            .mark_text(size=14, align="left", color="blue")
        )
    )
    .properties(
        width=180,
        height=180,
    )
    .facet(
        facet=alt.Facet(
            'site',
            title=None,
            header=alt.Header(
                titleFontSize=21,
                titleFontWeight='normal',
                titlePadding=5,
                labelFontSize=17,
                labelFontStyle='italic'
            )
        ),
        spacing=6,
        columns=4
    )
    .configure_axis(grid=False)
    .properties(
        title=alt.TitleParams(
            "Sites with high correlation between ACE2 affinity and sera escape",
            anchor="middle",
            fontSize=16,
        ),
    )
)
affinity_escape_corr_chart

alt.FacetChart(...)

## Plot sites with serum escape

In [14]:
escape_sites = [357,420, 440, 444, 452, 456, 473]

filtered_XBB15_df = filtered_XBB15_df[filtered_XBB15_df['site'].isin(escape_sites)]

init_min_func_effect = -1.5

func_effects_slider = alt.param(
    value=init_min_func_effect,
    bind=alt.binding_range(
        name="minimum spike-mediated entry",
        min=filtered_XBB15_df["spike mediated entry"].min(),
        max=0,
    ),
)

affinity_escape_corr_base = (
    alt.Chart(filtered_XBB15_df)
    .add_params(func_effects_slider)
    .transform_filter(alt.datum["spike mediated entry"] >= func_effects_slider)
)


sera_affinity_escape_corr_chart = (
    (
        (
            affinity_escape_corr_base
            .encode(
                x=alt.X("ACE2 binding", scale=alt.Scale(nice=False, padding=10)),
                y=alt.Y("human sera escape", scale=alt.Scale(nice=False, padding=10)),
                tooltip=filtered_XBB15_df.columns.tolist(),   
            )           
            .mark_circle(color="black", opacity=0.25, size=120)
        )
        + (
            affinity_escape_corr_base
            .transform_regression("ACE2 binding", "human sera escape", params=True)
            .transform_calculate(
                r=alt.expr.if_(
                    alt.datum["coef"][1] > 0,
                    alt.expr.sqrt(alt.datum["rSquared"]),
                    -alt.expr.sqrt(alt.datum["rSquared"]),
                ),
                r_text="r = " + alt.expr.format(alt.datum["r"], ".2f"),
            )
            .encode(
                text="r_text:N",
                x=alt.value(5),
                y=alt.value(170),
            )
            .mark_text(size=14, align="left", color="blue")
        )
    )
    .properties(
        width=180,
        height=180,
    )
    .facet(
        facet=alt.Facet(
            'site',
            title=None,
            header=alt.Header(
                titleFontSize=21,
                titleFontWeight='normal',
                titlePadding=5,
                labelFontSize=17,
                labelFontStyle='italic'
            )
        ),
        spacing=6,
        columns=4
    )
    .configure_axis(grid=False)
    .properties(
        title=alt.TitleParams(
            "Correlation of ACE2 binding and sera escape for sites targeted by antibodies ",
            anchor="middle",
            fontSize=16,
        ),
    )
)
sera_affinity_escape_corr_chart

alt.FacetChart(...)